In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [4]:
batch_size=64
train_dataloader=DataLoader(training_data,batch_size=batch_size)
test_dataloader=DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{X.shape}")
    print(f"Shape of y:{y.shape} {y.dtype}")
    break

Shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y:torch.Size([64]) torch.int64


In [5]:
device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [6]:

print(device)

cpu


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3)

In [9]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100==0:
            loss,current=loss.item(),(batch+1)*len(X)
            print(f"loss:{loss:>7f}[{current:>5d}|{size:>5d}]")

In [10]:
def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    model.eval()
    test_loss,correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device),y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
        test_loss/=num_batches
        correct/=size
        print(f"Test Error:\n Accuracy:{100*correct:>0.1f}%, Avg loss:{test_loss:>8f}\n")

In [11]:
epoches=5
for t in range(epoches):
    print(f"Epoch {t+1}\n-------------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
-------------------
loss:2.311336[   64|60000]
loss:2.289894[ 6464|60000]
loss:2.280707[12864|60000]
loss:2.275322[19264|60000]
loss:2.247575[25664|60000]
loss:2.229771[32064|60000]
loss:2.225733[38464|60000]
loss:2.204895[44864|60000]
loss:2.215563[51264|60000]
loss:2.165870[57664|60000]
Test Error:
 Accuracy:47.9%, Avg loss:2.164465

Epoch 2
-------------------
loss:2.183640[   64|60000]
loss:2.165344[ 6464|60000]
loss:2.116645[12864|60000]
loss:2.136365[19264|60000]
loss:2.076634[25664|60000]
loss:2.027184[32064|60000]
loss:2.046914[38464|60000]
loss:1.978704[44864|60000]
loss:1.994474[51264|60000]
loss:1.908785[57664|60000]
Test Error:
 Accuracy:51.2%, Avg loss:1.909425

Epoch 3
-------------------
loss:1.947505[   64|60000]
loss:1.910978[ 6464|60000]
loss:1.801199[12864|60000]
loss:1.848982[19264|60000]
loss:1.735637[25664|60000]
loss:1.686582[32064|60000]
loss:1.703799[38464|60000]
loss:1.604676[44864|60000]
loss:1.633357[51264|60000]
loss:1.525553[57664|60000]
Test Error

In [12]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model=NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
classes=[
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x,y=test_data[1][0],test_data[1][1]
with torch.no_grad():
    x=x.to(device)
    pred=model(x)
    predicted,actual=classes[pred[0].argmax(0)],classes[y]
    print(f'Precited:"{predicted}",Actual:"{actual}"')


Precited:"Pullover",Actual:"Pullover"
